# Tensorboard example #1: ANN

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime

print('complete')


complete


In [3]:
%load_ext tensorboard


Prerequisite: download data from [Kaggle](https://www.kaggle.com/datasets/krantiswalke/bankfullcsv)

In [4]:
data_path = '/Users/mikegoodman/Documents/developer/data/'

dataset = pd.read_csv(data_path + 'bank-full.csv', sep=',')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

dataset.head()

DIR = 'temp/'

In [5]:
# binary categorical variables

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) #marital
X[:, 4] = le.fit_transform(X[:, 4]) #default
X[:, 6] = le.fit_transform(X[:, 6]) #housing
X[:, 7] = le.fit_transform(X[:, 7]) #loan
y = le.fit_transform(y) #target column


In [6]:
# One hot encoding of other categorical variables (> 2 values)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1,3,8,10,15])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))


In [7]:
# train and test split

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [8]:
# scale data to unit variance

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train1 = sc.fit_transform(X_train1)
X_test1 = sc.transform(X_test1)


In [9]:
# remove existing logs, if applicable
#!rm -rf ./logs/

In [9]:
# define tensorboard callback for model
from tensorflow.keras.callbacks import TensorBoard 
import os

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = os.path.join(DIR, "model1/logs", datetime.now().strftime("%Y%m%d-%H%M%S")), histogram_freq=1)


In [10]:
# define an ANN model
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


2022-06-24 09:30:57.367413: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# compile and train the model
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train1, y_train1, batch_size = 32, epochs = 30, callbacks=[tensorboard_callback])


Epoch 1/30
1131/1131 [==============================] - 1s 875us/step - loss: 0.1819 - accuracy: 0.9167
Epoch 2/30
1131/1131 [==============================] - 1s 868us/step - loss: 0.1814 - accuracy: 0.9172
Epoch 3/30
1131/1131 [==============================] - 1s 869us/step - loss: 0.1810 - accuracy: 0.9179
Epoch 4/30
1131/1131 [==============================] - 1s 872us/step - loss: 0.1807 - accuracy: 0.9191
Epoch 5/30
1131/1131 [==============================] - 1s 880us/step - loss: 0.1806 - accuracy: 0.9174
Epoch 6/30
1131/1131 [==============================] - 1s 901us/step - loss: 0.1806 - accuracy: 0.9188
Epoch 7/30
1131/1131 [==============================] - 1s 885us/step - loss: 0.1798 - accuracy: 0.9188
Epoch 8/30
1131/1131 [==============================] - 1s 882us/step - loss: 0.1798 - accuracy: 0.9182
Epoch 9/30
1131/1131 [==============================] - 1s 898us/step - loss: 0.1797 - accuracy: 0.9189
Epoch 10/30
1131/1131 [==============================] - 1s 874u

In [13]:
# launch Tensorboard
%tensorboard --logdir $DIR/model1/logs


# Tensorboard example #2: Confusion matrix 

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
import sklearn

mnist = tf.keras.datasets.mnist

(X_train2, y_train2), (X_test2, y_test2) = mnist.load_data()
X_train2, X_test2 = X_train2 / 255.0, X_test2 / 255.0


In [15]:
class_names = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine']

In [16]:
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    threshold = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    return figure

In [18]:
from datetime import datetime

tensorboard_callback2 = tf.keras.callbacks.TensorBoard(log_dir = os.path.join(DIR, "model2/logs", datetime.now().strftime("%Y%m%d-%H%M%S")), histogram_freq=1)
file_writer_cm = tf.summary.create_file_writer(DIR + 'model2/logs/cm')

In [19]:
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)

    digit = tf.image.decode_png(buf.getvalue(), channels=4)
    digit = tf.expand_dims(digit, 0)

    return digit

In [20]:
from tensorflow import keras
from sklearn import metrics
import io

def log_confusion_matrix(epoch, logs):
    predictions = model.predict(X_test2)
    predictions = np.argmax(predictions, axis=1)

    cm = metrics.confusion_matrix(y_test2, predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [21]:
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

model = tf.keras.models.Sequential([
   tf.keras.layers.Flatten(input_shape=(28, 28)),
   tf.keras.layers.Dense(512, activation='relu'),
   tf.keras.layers.Dropout(0.2),
   tf.keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train2, y_train2, batch_size = 32, validation_split=0.2, epochs = 10, callbacks=[tensorboard_callback2, cm_callback])

Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.7029 - accuracy: 0.8207 - val_loss: 0.3663 - val_accuracy: 0.9023
Epoch 2/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3726 - accuracy: 0.8962 - val_loss: 0.2972 - val_accuracy: 0.9176
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3139 - accuracy: 0.9115 - val_loss: 0.2618 - val_accuracy: 0.9290
Epoch 4/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2787 - accuracy: 0.9207 - val_loss: 0.2368 - val_accuracy: 0.9351
Epoch 5/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2516 - accuracy: 0.9286 - val_loss: 0.2169 - val_accuracy: 0.9409
Epoch 6/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2301 - accuracy: 0.9345 - val_loss: 0.2034 - val_accuracy: 0.9459
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2135 - accuracy: 0.9401 - val_loss: 0.1891 - val_accuracy:

In [23]:
%tensorboard --logdir $DIR/model2/logs/

# Tensorboard example #3: Tensorboard Projector

In [24]:
from tensorboard.plugins import projector

In [25]:
print(X_train2[0].shape)

(28, 28)


In [26]:
file_writer = tf.summary.create_file_writer(DIR + 'model3/logs')

In [27]:
import numpy as np

with file_writer.as_default():
    image = np.reshape(X_train2[4], (-1, 28, 28, 1))
    tf.summary.image("Single Image", image, step=0)

In [30]:
%tensorboard --logdir $DIR/model3/logs


In [31]:
file_writer = tf.summary.create_file_writer(DIR + 'model3/logs/actual-images')

In [32]:
import numpy as np

with file_writer.as_default():
    images = np.reshape(X_train2[5:20], (-1, 28, 28, 1))
    tf.summary.image("Multiple Digits", images, max_outputs=16, step=0)

In [33]:
import io
import matplotlib.pyplot as plt

class_names = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine']

In [34]:
def image_grid():
    figure = plt.figure(figsize=(12,8))

    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.xlabel(class_names[y_train[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(X_train[i], cmap=plt.cm.coolwarm)

    return figure

In [35]:
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

In [38]:
%tensorboard --logdir $DIR/model3/logs/actual-images

# Tensorboard example #4: Hyperparameter tuning

In [39]:
from tensorboard.plugins.hparams import api as hp

In [40]:
logdir = DIR + "model4/logs/hparamas"

In [41]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 17]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1,0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

In [42]:
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(logdir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],)

In [43]:
def create_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS],  activation='relu'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation='softmax')])

    model.compile(optimizer=hparams[HP_OPTIMIZER],
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train2, y_train2, epochs=5)
    loss, accuracy = model.evaluate(X_test2, y_test2)

    return accuracy

In [44]:
def experiment(experiment_dir, hparams):

    with tf.summary.create_file_writer(experiment_dir).as_default():
        hp.hparams(hparams)
        accuracy = create_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [45]:
experiment_no = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,}

            experiment_name = f'Experiment {experiment_no}'
            print(f'Starting Experiment: {experiment_name}')
            print({h.name: hparams[h] for h in hparams})
            experiment(logdir + '/' + experiment_name, hparams)
            experiment_no += 1

Starting Experiment: Experiment 0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/5
1875/1875 [==============================] - 2s 816us/step - loss: 0.5642 - accuracy: 0.8300
Epoch 2/5
1875/1875 [==============================] - 2s 852us/step - loss: 0.3481 - accuracy: 0.8959
Epoch 3/5
1875/1875 [==============================] - 2s 933us/step - loss: 0.3123 - accuracy: 0.9047
Epoch 4/5
1875/1875 [==============================] - 2s 831us/step - loss: 0.2896 - accuracy: 0.9127
Epoch 5/5
313/313 [==============================] - 0s 627us/step - loss: 0.1997 - accuracy: 0.9410
Starting Experiment: Experiment 1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'rmsprop'}
Epoch 1/5
1875/1875 [==============================] - 2s 851us/step - loss: 0.5486 - accuracy: 0.8384
Epoch 2/5
1875/1875 [==============================] - 2s 801us/step - loss: 0.3542 - accuracy: 0.8942
Epoch 3/5
1875/1875 [==============================] - 2s 815us/step - loss: 0.3212 - accuracy: 0.90

In [46]:
%tensorboard --logdir $DIR/model4/logs

# Tensorboard example #5: Profiler (profile model computation performance)

In [49]:
#!pip install -U tensorboard_plugin_profile

In [50]:
device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [51]:
logdir = DIR + "model5/logs"

In [52]:
callbacks = [TensorBoard(log_dir=logdir, profile_batch='10,20')]

2022-06-24 09:43:17.346132: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-24 09:43:17.346151: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-24 09:43:17.348633: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


In [53]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [54]:
model.fit(X_train2, y_train2, epochs=10, validation_split=0.2, callbacks=callbacks)

Epoch 1/10
  65/1500 [>.............................] - ETA: 4s - loss: 1.9957 - accuracy: 0.3952

2022-06-24 09:43:31.465327: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-24 09:43:31.465345: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-24 09:43:31.501790: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-24 09:43:31.509511: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-24 09:43:31.515196: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: temp/model5/logs/plugins/profile/2022_06_24_09_43_31

2022-06-24 09:43:31.518796: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to temp/model5/logs/plugins/profile/2022_06_24_09_43_31/mikegoodman-macbookpro3220520.roam.corp.google.com.trace.json.gz
2022-06-24 09:43:31.523194: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: temp/model5/logs/plugins/pr

1500/1500 [==============================] - 6s 3ms/step - loss: 0.6977 - accuracy: 0.8226 - val_loss: 0.3651 - val_accuracy: 0.9047
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3704 - accuracy: 0.8958 - val_loss: 0.2952 - val_accuracy: 0.9193
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3131 - accuracy: 0.9100 - val_loss: 0.2611 - val_accuracy: 0.9277
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2775 - accuracy: 0.9208 - val_loss: 0.2385 - val_accuracy: 0.9346
Epoch 5/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2522 - accuracy: 0.9282 - val_loss: 0.2188 - val_accuracy: 0.9394
Epoch 6/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2328 - accuracy: 0.9347 - val_loss: 0.2037 - val_accuracy: 0.9451
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2148 - accuracy: 0.9403 - val_loss: 0.1919 - val_accuracy: 0.9482
Epo

In [55]:
%tensorboard --logdir $DIR/model5/logs

# Tensorboard example #6: Deep Learning Frameworks (WIP)

In [ ]:
from tensorboardX import SummaryWriter
import xgboost as xgb

# read in data
dtrain = xgb.DMatrix(X_train1, y_train1)
dtest = xgb.DMatrix(X_test1)

# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2

def TensorBoardCallback():
    writer = SummaryWriter()

    def callback(env):
        for k, v in env.evaluation_result_list:
            writer.add_scalar(k, v, env.iteration)

    return callback

bst = xgb.train(param, dtrain, num_round, callbacks=[TensorBoardCallback()])

# make prediction
preds = bst.predict(dtest)
preds

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir='logs')

from torch.utils.tensorboard import SummaryWriter
import numpy as np

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [ ]:
# run in terminal to create tensorboard.dev project
tensorboard dev upload --logdir logs \
--name "(optional) My latest experiment" \
--description "(optional) Simple comparison of several hyperparameters"

# run in terminal to revoke access
tensorboard dev auth revoke

# Tensorboard example #7: Tensorboard with Vertex AI (WIP)

In [ ]:
## complete section and upload to git
# https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview